# Imports

In [247]:
import pandas as pd
import numpy as np
import datetime as dt

In [138]:
import warnings
warnings.filterwarnings('ignore')


In [885]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.options.display.max_colwidth = 500
pd.options.display.float_format ='{:.2f}'.format

In [13]:
def end_of_client(lt):
    n = 0
    while lt[n] != 0: n = n+1
    lt[n] = 1
    return lt

In [775]:
def data_up():
    global CLDF, OPDF
    CLDF = pd.read_csv(r'C:\Users\dmivadivanov\Documents\ProChat\Data\CustomerDF.csv', sep = ';')
    CLDF['start_time'] = pd.to_datetime(CLDF['start_time'])
    #CLDF['time_predict'] = pd.to_timedelta(CLDF['time_predict'], unit = 's')
    CLDF['operator_id'] = 0
    CLDF['day_part'] = 'lanch'
    CLDF['end'] = 0
    part1 = pd.to_datetime('12:00:00')
    part2 = pd.to_datetime('17:30:00')
    CLDF.loc[CLDF['start_time'] < part1, 'day_part'] = 'morning'
    CLDF.loc[CLDF['start_time'] > part2, 'day_part'] = 'evening'
    CLDF['wait_ready'] = 0
    CLDF['late'] = 0

    for i in range(1, CLDF.shape[0]+1):
        CLDF.loc[CLDF['id'] == i, 'wait_ready'] =\
        Customer_time_to_wait.loc[Customer_time_to_wait['day_part'] == CLDF.loc[CLDF['id'] == i, 'day_part'].item(), 
                              CLDF.loc[CLDF['id'] == i, 'task_type'].item()].item()
    OPDF = pd.read_csv(r'C:\Users\dmivadivanov\Documents\ProChat\Data\OperatorDF.csv', sep = ';')
    OPDF['list_of_clien'] = ''
    OPDF['end_time_predict'] = pd.to_datetime('8:00:00')
    OPDF['client_count'] = 0
    OPDF['hard_sum'] = 0
    print()

# Data

In [550]:
CLDF = pd.read_csv(r'C:\Users\dmivadivanov\Documents\ProChat\Data\CustomerDF.csv', sep = ';')
CLDF['start_time'] = pd.to_datetime(CLDF['start_time'])
#CLDF['time_predict'] = pd.to_timedelta(CLDF['time_predict'], unit = 's')
CLDF['operator_id'] = 0
CLDF.head(1)

,id,start_time,end,psi,task_type,operator_id
0,1,2022-12-18 08:00:00,0,2,w1,0


In [ ]:
CLDF['day_part'] = 'lanch'
CLDF.loc[CLDF['start_time'] < pd.to_datetime('2022-12-17 12:00:00').to_datetime64(), 'day_part'] = 'morning'
CLDF.loc[CLDF['start_time'] > pd.to_datetime('2022-12-17 17:30:00'), 'day_part'] = 'evening'
CLDF['wait_ready'] = 0
for i in range(1, CLDF.shape[0]+1):
    CLDF.loc[CLDF['id'] == i, 'wait_ready'] =\
    Customer_time_to_wait.loc[Customer_time_to_wait['day_part'] == CLDF.loc[CLDF['id'] == i, 'day_part'].item(), 
                              CLDF.loc[CLDF['id'] == i, 'task_type'].item()].item()

In [287]:
OPDF = pd.read_csv(r'C:\Users\dmivadivanov\Documents\ProChat\Data\OperatorDF.csv', sep = ';')
OPDF['list_of_clien'] = ''
OPDF['end_time_predict'] = pd.to_datetime('2022-12-17 8:00:00')
OPDF.head(1)

,id,psi,list_of_clien,end_time_predict
0,1,1,,2022-12-17 08:00:00


In [288]:
psi_matrix = pd.read_csv(r'C:\Users\dmivadivanov\Documents\ProChat\Data\Psi_Matrix.csv', sep = ';')

In [289]:
Operator_time_predict = pd.read_csv(r'C:\Users\dmivadivanov\Documents\ProChat\Data\Operators_time_predict.csv', sep = ';')

In [290]:
Customer_time_to_wait = pd.read_csv(r'C:\Users\dmivadivanov\Documents\ProChat\Data\Customer_time_to_wait.csv', sep = ';')

# Data Podgotovka

In [293]:
CLDF['day_part'] = 'lanch'
CLDF.loc[CLDF['start_time'] < pd.to_datetime('2022-12-17 12:00:00').to_datetime64(), 'day_part'] = 'morning'
CLDF.loc[CLDF['start_time'] > pd.to_datetime('2022-12-17 17:30:00'), 'day_part'] = 'evening'
CLDF['wait_ready'] = 0
for i in range(1, CLDF.shape[0]+1):
    CLDF.loc[CLDF['id'] == i, 'wait_ready'] =\
    Customer_time_to_wait.loc[Customer_time_to_wait['day_part'] == CLDF.loc[CLDF['id'] == i, 'day_part'].item(), 
                              CLDF.loc[CLDF['id'] == i, 'task_type'].item()].item()

In [294]:
CLDF['wait_ready'] = 0
for i in range(1, CLDF.shape[0]+1):
    CLDF.loc[CLDF['id'] == i, 'wait_ready'] =\
    Customer_time_to_wait.loc[Customer_time_to_wait['day_part'] == CLDF.loc[CLDF['id'] == i, 'day_part'].item(), 
                              CLDF.loc[CLDF['id'] == i, 'task_type'].item()].item()

In [295]:
CLDF

,id,start_time,time_predict,end,psi,task_type,operator_id,day_part,wait_ready
0,1,2022-12-17 08:00:00,0 days 00:02:00,0,2,w1,0,morning,267
1,2,2022-12-17 08:02:00,0 days 00:12:00,0,3,w2,0,morning,250
2,3,2022-12-17 08:04:00,0 days 00:14:00,0,1,w3,0,morning,136
3,4,2022-12-17 08:06:00,0 days 00:13:00,0,2,w4,0,morning,281
4,5,2022-12-17 08:08:00,0 days 00:11:00,0,3,w1,0,morning,267
...,...,...,...,...,...,...,...,...,...
236,237,2022-12-17 15:52:00,0 days 00:10:00,0,3,w1,0,lanch,415
237,238,2022-12-17 15:54:00,0 days 00:06:00,0,1,w2,0,lanch,251
238,239,2022-12-17 15:56:00,0 days 00:02:00,0,2,w3,0,lanch,111
239,240,2022-12-17 15:58:00,0 days 00:15:00,0,1,w4,0,lanch,321


# Algorytm

In [268]:
data_up()
start_of_day = pd.to_datetime('8:00:00')
for time in range (0, 99):
    now_time = start_of_day + pd.to_timedelta(time, unit = 's')
    for check in range(0, len(OPDF['id'])):
        if OPDF['end_time_predict'][check] == now_time:
            OPDF.iloc[check, 3] = OPDF.iloc[check, 3] + pd.to_timedelta(CLDF[(CLDF['end'] == 0) & (CLDF['start_time'] < now_time)].head(1)['time_predict'], unit = 's')
            OPDF.iloc[check, 2] =  OPDF.iloc[check, 2] + '|' + str(CLDF[(CLDF['end'] == 0) & (CLDF['start_time'] < now_time)].head(1).iloc[0,0])
            new_id = CLDF[(CLDF['end'] == 0) & (CLDF['start_time'] < now_time)].head(1).iloc[0,0]
            CLDF.end = end_of_client(CLDF.end)
            CLDF.loc[CLDF['id'] == new_id, 'operator_id' ] = OPDF.id[check]

ValueError: Must have equal len keys and value when setting with an iterable

In [30]:
OPDF.head(20)

,id,psi,list_of_clien,end_time_predict
0,1,1,|1|11|17|28|34|38|48|66|72|75|83|97|105|109|122|131|135,2022-12-10 09:48:00
1,2,2,|2|18|24|25|42|47|54|64|79|86|89|99|116|121,2022-12-10 09:42:00
2,3,3,|3|22|30|41|50|51|71|88|94|108|110|113|119|125|132,2022-12-10 09:51:00
3,4,2,|4|20|26|35|45|49|60|73|82|91|106|111|133,2022-12-10 09:43:00
4,5,1,|5|14|36|52|67|69|80|92|101|120|128,2022-12-10 09:46:00
5,6,2,|6|12|15|23|37|39|53|68|70|84|95|117|126|134,2022-12-10 09:43:00
6,7,3,|7|16|27|32|43|55|62|74|76|87|102|114|129,2022-12-10 09:42:00
7,8,1,|8|13|29|31|46|56|65|81|93|103|118|123|130,2022-12-10 09:47:00
8,9,1,|9|21|40|58|63|77|85|96|100|107|112|127,2022-12-10 09:44:00
9,10,2,|10|19|33|44|57|59|61|78|90|98|104|115|124,2022-12-10 09:39:00


### Давайте выбирать по самому маленькому весу

In [364]:
data_up()

In [33]:
data_up()
start_of_day = pd.to_datetime('8:00:00')
for time in range (0, 150):
    now_time = start_of_day + pd.to_timedelta(time, unit = 's')
    for check in range(0, len(OPDF['id'])):
#проверяем на наличие освободившегося оператора
        if OPDF['end_time_predict'][check] == now_time:
            
#меняем время освобождения у первого свободного оператора
            OPDF.iloc[check, 3] = OPDF.iloc[check, 3] + \
                pd.to_timedelta(CLDF[CLDF['end'] == 0].head(1)['time_predict'], unit = 's')
            
#записываем первому свободному оператору первого свободного клиента
            OPDF.iloc[check, 2] =  OPDF.iloc[check, 2] + '|' + str(CLDF[CLDF['end'] == 0].head(1).iloc[0,0])

#Узнаем id первого освбодившегося клиента (= выбранного нами)
            new_id = CLDF[CLDF['end'] == 0].head(1).iloc[0,0]
    
#ставим первому свободному клиенту статус НЕ СВОБОДЕН
            CLDF.end = end_of_client(CLDF.end)б
    
#записываем этому клиенту ID оператора, котрые его обработал
            CLDF.loc[CLDF['id'] == new_id, 'operator_id' ] = OPDF.id[check]

C:\Users\dmivadivanov\AppData\Local\Temp\ipykernel_10436\2402569277.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lt[n] = 1


### Создаём вес g1

In [440]:
OP1 = OPDF.head(1)
OP1

,id,psi,list_of_clien,end_time_predict
0,1,1,,2022-12-18 08:00:00


In [441]:
CL1 = CLDF.head(10)

In [551]:
def get_g1(df):
    global psi_matrix, FreeOp
    df['g1'] = 0
    for i in df.id:
        df.loc[df['id'] == i, 'g1'] = 10 * psi_matrix.loc[psi_matrix['psi'] == df.loc[df['id'] == i, 'psi'].item(),
                                                          f'{FreeOp.psi.item()}'].item()
    return df

In [467]:
CL1 = get_g1(CL1)

In [468]:
CL1

,id,start_time,time_predict,end,psi,task_type,operator_id,day_part,wait_ready,g1
0,1,2022-12-18 08:00:00,0 days 00:02:00,0,2,w1,0,evening,475,10
1,2,2022-12-18 08:02:00,0 days 00:12:00,0,3,w2,0,evening,138,5
2,3,2022-12-18 08:04:00,0 days 00:14:00,0,1,w3,0,evening,195,5
3,4,2022-12-18 08:06:00,0 days 00:13:00,0,2,w4,0,evening,446,10
4,5,2022-12-18 08:08:00,0 days 00:11:00,0,3,w1,0,evening,475,5
5,6,2022-12-18 08:10:00,0 days 00:02:00,0,1,w2,0,evening,138,5
6,7,2022-12-18 08:12:00,0 days 00:11:00,0,1,w3,0,evening,195,5
7,8,2022-12-18 08:14:00,0 days 00:04:00,0,1,w4,0,evening,446,5
8,9,2022-12-18 08:16:00,0 days 00:13:00,0,3,w1,0,evening,475,5
9,10,2022-12-18 08:18:00,0 days 00:12:00,0,1,w2,0,evening,138,0


### Создаём вес g2

In [590]:
def get_g2(df):
    global Operator_time_predict, FreeOp
    df['g2'] = 0
    df['work_time_predict'] = ''
    for i in df.id:
        df.loc[df['id'] == i, 'work_time_predict'] =\
        Operator_time_predict.loc[Operator_time_predict['id'] == FreeOp.id.item(), 
                                  df.loc[df['id'] == i, 'task_type'].item()].item()
    df['g2'] = df['work_time_predict']
    return df

In [213]:
CL1['g2*'] = np.log(CL1['g2'])
CL1['g_sum'] = (CL1['g1'] + CL1['g2*'])/2
CL1['g_sum'].max()
CL1[CL1['g_sum'] == CL1['g_sum'].max()].head(1) #<--- нашли того клиента, которого надо назначить освободившемуся клиенту

,id,start_time,time_predict,end,psi,task_type,operator_id,g1,g2,g2*,g_sum
2,3,2022-12-16 08:04:00,0 days 00:14:00,0,1,w3,0,10.0000,327,5.7900,7.8950


### Создаём вес g3

In [353]:
now = pd.to_datetime('08:20:00')

In [554]:
def get_g3(df):
    global Operator_time_predict, FreeOp, now_time
    df['g3'] = 0
    for i in df.id:
        df.loc[df['id'] == i, 'g3'] =\
        df.loc[df['id'] == i, 'wait_ready'].item()\
        - ((now_time - pd.to_datetime(df.start_time)) / pd.Timedelta(seconds=1))\
        - Operator_time_predict.loc[Operator_time_predict['id'] == FreeOp.id.item(),
                                df.loc[df['id'] == i, 'task_type'].item()].item()
    return df

### Считаем критический коэффициент

In [636]:
WaitClient

,id,start_time,end,psi,task_type,operator_id,day_part,wait_ready,g1,g2,work_time_predict,g3
2,3,2022-12-18 08:00:00,0,1,w3,0,morning,136,2.5000,118,118,18
3,4,2022-12-18 08:00:00,0,2,w4,0,morning,281,5.0000,462,462,-181


In [643]:
def get_KST(df):
    global Operator_time_predict, FreeOp, now_time
    for i in df.id:
        df.loc[df['id'] == i,'K_stock'] =\
                    (df['start_time'] + pd.to_timedelta( WaitClient['wait_ready'] , unit = 's') - now_time)\
                    / np.timedelta64(1, 's') \
                    / Operator_time_predict.loc[Operator_time_predict['id'] == FreeOp.head(1)['id'].item(),
                                      WaitClient.loc[WaitClient['id'] == i, 'task_type'].item()].item()
    return df

In [646]:
get_KST(WaitClient)

AttributeError: 'NoneType' object has no attribute 'id'

In [647]:
WaitClient

In [356]:
CL1 = get_g3(CL1)

In [360]:
CL1[CL1['g_sum'] == CL1['g_sum'].max()].head(1) #<--- нашли того клиента, которого надо назначить освободившемуся клиенту

,id,start_time,time_predict,end,psi,task_type,operator_id,day_part,wait_ready,g1,g2,g3,g_sum
9,10,2022-12-17 08:18:00,0 days 00:12:00,0,1,w2,0,morning,250,0.0000,227,-97,43.3333


## Делаем общий прогон

In [704]:
CLDF['end_time_predict'] = pd.to_datetime('8:00:00')

In [705]:
CLDF

,id,start_time,end,psi,task_type,operator_id,day_part,wait_ready,end_time_predict
0,1,2022-12-18 08:00:00,1,2,w1,4,morning,267,2022-12-18 08:00:00
1,2,2022-12-18 08:00:00,1,3,w2,2,morning,250,2022-12-18 08:00:00
2,3,2022-12-18 08:00:00,1,1,w3,1,morning,136,2022-12-18 08:00:00
3,4,2022-12-18 08:00:00,1,2,w4,3,morning,281,2022-12-18 08:00:00
4,5,2022-12-18 08:01:10,1,3,w1,5,morning,267,2022-12-18 08:00:00
...,...,...,...,...,...,...,...,...,...
236,237,2022-12-18 08:39:50,0,3,w1,0,morning,267,2022-12-18 08:00:00
237,238,2022-12-18 08:40:00,0,1,w2,0,morning,250,2022-12-18 08:00:00
238,239,2022-12-18 08:40:10,0,2,w3,0,morning,136,2022-12-18 08:00:00
239,240,2022-12-18 08:40:20,0,1,w4,0,morning,281,2022-12-18 08:00:00


In [874]:
%%time
data_up()
start_of_day = pd.to_datetime('8:00:00')
l1 = 0.3
l2 = 0.3
l3 = 1 - l1 - l2
minutes = 100
#while (CLDF.end.sum() != CLDF.shape[0]) & (time != 60*minutes - 1):
for time in range (0, 60 * minutes):
        now_time = start_of_day + pd.to_timedelta(time, unit = 's')
        for check in OPDF.id:
                #проверяем на наличие освободившегося оператора
                if OPDF.loc[OPDF['id'] == check,'end_time_predict'].item() <= now_time:
                    #Вырезаем свободного оператора
                    FreeOp = OPDF.loc[OPDF['id'] == check, :]
                    #Вырезаем неназначенных клиентов 
                    WaitClient = CLDF[(CLDF['end'] == 0) & (CLDF['start_time'] <= now_time)]
                    if WaitClient.shape[0]>0:
                        #Раскидаем веса
                        WaitClient = get_g1(WaitClient)
                        WaitClient = get_g2(WaitClient)
                        WaitClient = get_g3(WaitClient)
                        WaitClient = get_KST(WaitClient)
                        WaitClient['g_sum'] = WaitClient['K_stock']*(l1*WaitClient['g1'] + 
                                                                     l2*WaitClient['g2'] + 
                                                                     l3*WaitClient['g3'])
                        #Ищем клиента с самым маленьким весом
                        BestID = WaitClient[WaitClient['g_sum'] == WaitClient['g_sum'].min()].head(1)['id'].item()
                        #<--- нашли того клиента, которого надо назначить освободившемуся клиенту
                        
                        #меняем время освобождения у подходящего свободного оператора
                            #ВОТ ТУТ НАДО БУДЕТ ДОПОЛНИТЬ, ЧТО МЫ ПРОГНОЗИРУЕМ ПО ПРИМЕРНОМУ ВРЕМЕНИ
                            #НО ВСТАВЛЯЕМ СРАЗУ РЕАЛЬНОЕ
                        OPDF.loc[OPDF['id'] == check, 'end_time_predict'] = \
                        OPDF.loc[OPDF['id'] == check, 'end_time_predict'] +\
                        pd.to_timedelta(WaitClient[WaitClient['id'] == BestID].head(1)['work_time_predict'].item(), unit = 's')
                          
                        #ставим клиенту время окончания разговора
                        CLDF.loc[CLDF['id'] == BestID, 'end_time_predict'] = OPDF.loc[OPDF['id'] == check, 'end_time_predict'].item()                        
                        
                        CLDF.loc[CLDF['id'] == BestID, 'match'] = 10 * psi_matrix.loc[psi_matrix['psi'] == CLDF.loc[CLDF['id'] == BestID, 'psi'].item(),
                                                              f'''{FreeOp.head(1)['psi'].item()}'''].item()
                        #записываем свободному оператору подходящего клиента
                        OPDF.loc[OPDF['id'] == check, 'list_of_clien'] =\
                        OPDF.loc[OPDF['id'] == check, 'list_of_clien'] + '|' + str(BestID)
                    
                        #записываем выбранному оператору количество клиентов и их суммарную сложность
                        OPDF.loc[OPDF['id'] == check, 'client_count'] += 1
                        OPDF.loc[OPDF['id'] == check, 'hard_sum'] += CLDF.loc[CLDF['id'] == BestID, 'hard_level'].item()
    
                        #ставим выбранному клиенту статус НЕ СВОБОДЕН
                        CLDF.loc[CLDF['id'] == BestID, 'end'] = 1
                    
                        #записываем этому клиенту ID оператора, котрые его обработал
                        CLDF.loc[CLDF['id'] == BestID, 'operator_id' ] = check
CLDF['deadline'] = CLDF['start_time'] + pd.to_timedelta(CLDF['wait_ready'], unit = 's')
for i in CLDF.id:
    CLDF.loc[CLDF['id']==i, 'late'] = max(0,  (CLDF.loc[CLDF['id'] == i, 'end_time_predict'].item() - CLDF.loc[CLDF['id'] == i, 'deadline'].item()) / np.timedelta64(1, 's')  )
f1 = CLDF.late.mean() 
f2 = CLDF.match.mean()
OPDF['tired'] = OPDF['hard_sum'] / OPDF['client_count'] / OPDF['h_lvl']
f3 = OPDF.tired.mean()
results = results.append(pd.DataFrame([[l1, l2, l3, f1, f2, f3]],columns=['L1', 'L2', 'L3', 'f1', 'f2', 'f3']))


Wall time: 34.3 s


In [875]:
CLDF

,id,start_time,psi,task_type,hard_level,operator_id,day_part,end,wait_ready,late,end_time_predict,match
0,1,2022-12-19 08:00:00,2,w1,2,4,morning,1,267,0,2022-12-19 08:08:00,10.0000
1,2,2022-12-19 08:00:00,3,w2,5,2,morning,1,250,0,2022-12-19 08:06:01,5.0000
2,3,2022-12-19 08:00:00,1,w3,6,1,morning,1,136,0,2022-12-19 08:05:27,10.0000
3,4,2022-12-19 08:00:00,2,w4,1,3,morning,1,281,0,2022-12-19 08:07:42,5.0000
4,5,2022-12-19 08:01:10,3,w1,5,5,morning,1,267,0,2022-12-19 08:04:17,2.5000
5,6,2022-12-19 08:01:20,1,w2,8,6,morning,1,250,0,2022-12-19 08:05:17,5.0000
6,7,2022-12-19 08:01:30,1,w3,8,7,morning,1,136,0,2022-12-19 08:05:52,2.5000
7,8,2022-12-19 08:01:40,1,w4,4,8,morning,1,281,0,2022-12-19 08:03:40,10.0000
8,9,2022-12-19 08:01:50,3,w1,4,9,morning,1,267,0,2022-12-19 08:01:40,2.5000
9,10,2022-12-19 08:02:00,1,w2,9,9,morning,1,250,0,2022-12-19 08:04:54,10.0000


## Считаем целевые функции

In [886]:
results = pd.DataFrame(np.array([[0, 0, 0, 0, 0, 0]]),
                   columns=['L1', 'L2', 'L3', 'f1', 'f2', 'f3'])

In [888]:
%%time
for l1 in range(0, 11):
    for l2 in range(0, 11):
        data_up()
        start_of_day = pd.to_datetime('8:00:00')
        l1 = l1/10
        l2 = l2/10
        l3 = 1 - l1 - l2
        minutes = 100
        #while (CLDF.end.sum() != CLDF.shape[0]) & (time != 60*minutes - 1):
        for time in range (0, 60 * minutes):
                now_time = start_of_day + pd.to_timedelta(time, unit = 's')
                for check in OPDF.id:
                        #проверяем на наличие освободившегося оператора
                        if OPDF.loc[OPDF['id'] == check,'end_time_predict'].item() <= now_time:
                            #Вырезаем свободного оператора
                            FreeOp = OPDF.loc[OPDF['id'] == check, :]
                            #Вырезаем неназначенных клиентов 
                            WaitClient = CLDF[(CLDF['end'] == 0) & (CLDF['start_time'] <= now_time)]
                            if WaitClient.shape[0]>0:
                                #Раскидаем веса
                                WaitClient = get_g1(WaitClient)
                                WaitClient = get_g2(WaitClient)
                                WaitClient = get_g3(WaitClient)
                                WaitClient = get_KST(WaitClient)
                                WaitClient['g_sum'] = WaitClient['K_stock']*(l1*WaitClient['g1'] + 
                                                                             l2*WaitClient['g2'] + 
                                                                             l3*WaitClient['g3'])
                                #Ищем клиента с самым маленьким весом
                                BestID = WaitClient[WaitClient['g_sum'] == WaitClient['g_sum'].min()].head(1)['id'].item()
                                #<--- нашли того клиента, которого надо назначить освободившемуся клиенту
                                
                                #меняем время освобождения у подходящего свободного оператора
                                    #ВОТ ТУТ НАДО БУДЕТ ДОПОЛНИТЬ, ЧТО МЫ ПРОГНОЗИРУЕМ ПО ПРИМЕРНОМУ ВРЕМЕНИ
                                    #НО ВСТАВЛЯЕМ СРАЗУ РЕАЛЬНОЕ
                                OPDF.loc[OPDF['id'] == check, 'end_time_predict'] = \
                                OPDF.loc[OPDF['id'] == check, 'end_time_predict'] +\
                                pd.to_timedelta(WaitClient[WaitClient['id'] == BestID].head(1)['work_time_predict'].item(), unit = 's')
                                  
                                #ставим клиенту время окончания разговора
                                CLDF.loc[CLDF['id'] == BestID, 'end_time_predict'] = OPDF.loc[OPDF['id'] == check, 'end_time_predict'].item()                        
                                
                                CLDF.loc[CLDF['id'] == BestID, 'match'] = 10 * psi_matrix.loc[psi_matrix['psi'] == CLDF.loc[CLDF['id'] == BestID, 'psi'].item(),
                                                                      f'''{FreeOp.head(1)['psi'].item()}'''].item()
                                #записываем свободному оператору подходящего клиента
                                OPDF.loc[OPDF['id'] == check, 'list_of_clien'] =\
                                OPDF.loc[OPDF['id'] == check, 'list_of_clien'] + '|' + str(BestID)
                            
                                #записываем выбранному оператору количество клиентов и их суммарную сложность
                                OPDF.loc[OPDF['id'] == check, 'client_count'] += 1
                                OPDF.loc[OPDF['id'] == check, 'hard_sum'] += CLDF.loc[CLDF['id'] == BestID, 'hard_level'].item()
            
                                #ставим выбранному клиенту статус НЕ СВОБОДЕН
                                CLDF.loc[CLDF['id'] == BestID, 'end'] = 1
                            
                                #записываем этому клиенту ID оператора, котрые его обработал
                                CLDF.loc[CLDF['id'] == BestID, 'operator_id' ] = check
        CLDF['deadline'] = CLDF['start_time'] + pd.to_timedelta(CLDF['wait_ready'], unit = 's')
        for i in CLDF.id:
            CLDF.loc[CLDF['id']==i, 'late'] = max(0,  (CLDF.loc[CLDF['id'] == i, 'end_time_predict'].item() - CLDF.loc[CLDF['id'] == i, 'deadline'].item()) / np.timedelta64(1, 's')  )
        f1 = CLDF.late.mean() 
        f2 = CLDF.match.mean()
        OPDF['tired'] = OPDF['hard_sum'] / OPDF['client_count'] / OPDF['h_lvl']
        f3 = OPDF.tired.mean()
        results = results.append(pd.DataFrame([[l1, l2, l3, f1, f2, f3]],columns=['L1', 'L2', 'L3', 'f1', 'f2', 'f3']))

KeyboardInterrupt: 

In [889]:
results

,L1,L2,L3,f1,f2,f3
0,0.00,0.00,0.00,0.00,0.00,0.00
0,0.00,0.00,1.00,1293.30,6.20,0.75
0,0.00,0.10,0.90,1304.01,6.15,0.75
0,0.00,0.20,0.80,1314.89,5.85,0.74
0,0.00,0.30,0.70,1282.14,6.15,0.74
0,0.00,0.40,0.60,1290.23,5.88,0.75
0,0.00,0.50,0.50,1171.51,5.88,0.74
0,0.00,0.60,0.40,1297.16,5.90,0.74
0,0.00,0.70,0.30,1281.96,6.05,0.76
0,0.00,0.80,0.20,1263.31,6.12,0.74
